# 5: Test Case Retrieval With Local Methods

In [2]:
import pandas as pd
import numpy as np
import pickle
import time

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import KNeighborsClassifier

from collections import Counter

from copy import deepcopy

from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras import backend as K

# My own algorithms
from Weighted_KNN_Classifier import LocalWeightedKNN

In [3]:
df = pd.read_csv("processed_df.csv")
feature_names = df.columns

## Load Data Etc.

In [4]:
k = 1

In [5]:
knn_clf = pickle.load(open("k-nn_model.sav", 'rb'))
model = load_model("NN.h5")

In [6]:
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

In [9]:
X_train_grad_LIME = np.load("X_train_grad_LIME.npy")
X_test_grad_LIME = np.load("X_test_grad_LIME.npy")

X_train_intgrad = np.load("X_train_intgrad.npy")
X_test_intgrad = np.load("X_test_intgrad.npy")
X_train_deeplift = np.load("X_train_deeplift.npy")
X_test_deeplift = np.load("X_test_deeplift.npy")
X_train_lrp = np.load("X_train_lrp.npy")
X_test_lrp = np.load("X_test_lrp.npy")

## Contributions Search - Novel Method
Search for similar cases based on the contributions of each feature, <i>not</i> their feature values.

In [10]:
nn_pred = model.predict_classes(X_test)

In [11]:
techniques = [["k-NN*", X_train, X_test],
              ["C-DeepLIFT", X_train_deeplift, X_test_deeplift],
              ["C-Integraded Gradients", X_train_intgrad, X_test_intgrad],
              ["C-LRP", X_train_lrp, X_test_lrp]]

for item in techniques:
    
    technique = item[0]
    train = item[1]
    test = item[2]

    print("==================================================")
    print(technique)
    print("==================================================")

    kNN = KNeighborsClassifier(n_neighbors=k, algorithm="brute") 
    kNN.fit(train, y_train)

    knn_predictions_test = kNN.predict(test)
    print("Accuracy:", accuracy_score(y_test, knn_predictions_test))
    print(confusion_matrix(y_test, knn_predictions_test, labels=None, sample_weight=None))
    print(" ")
    
    right = 0
    for i in range(len(nn_pred)):
        if knn_predictions_test[i] == nn_pred[i]:
            right += 1

    print("Agreement:", right/len(nn_pred))

k-NN*
Accuracy: 0.732
[[3852  835]
 [ 773  540]]
 
Agreement: 0.8168333333333333
C-DeepLIFT
Accuracy: 0.7406666666666667
[[3917  770]
 [ 786  527]]
 
Agreement: 0.8241666666666667
C-Integraded Gradients
Accuracy: 0.7438333333333333
[[3915  772]
 [ 765  548]]
 
Agreement: 0.8256666666666667
C-LRP
Accuracy: 0.74
[[3914  773]
 [ 787  526]]
 
Agreement: 0.8295


## Local Linear Model - Nugent and Cunningham
We can use the values to weigh a k-NN search across normal euclidean space.

$Distance(x,q) = \sqrt{\sum_{f=1}^{n}|w_{f}|Difference(x_f, q_f)^2}$

In [12]:
weighted_knn = LocalWeightedKNN()
weighted_knn.fit(X_train, y_train, k=k)

In [14]:
techniques = [["Local Linear Model", X_test_grad_LIME[:, :len(feature_names)]]]

for item in techniques:
    
    technique = item[0]
    weights = item[1]

    print("==================================================")
    print(technique)
    print("==================================================")

    start = time.time()
    predictions = list()

    for i in range(len(X_test)):            
        prediction = weighted_knn.predict(X_test[i], abs(weights[i]))
        predictions.append(prediction)

    print("Accuracy: ", accuracy_score(y_test, predictions))
    print(confusion_matrix(y_test, predictions, labels=None, sample_weight=None))
    print("Time taken:", time.time() - start)
    print(" ")
    
    right = 0
    for i in range(len(nn_pred)):
        if predictions[i] == nn_pred[i]:
            right += 1

    print("Agreement:", right/len(nn_pred))

Local Linear Model
Accuracy:  0.7373333333333333
[[3899  788]
 [ 788  525]]
Time taken: 1102.377187013626
 
Agreement: 0.8285
